In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import torch.nn.functional as F

In [ ]:
from unet.unet import DDPM
from loss_func.loss import Loss
from diffusion.diffusion import GaussianDiffusion

In [ ]:
from data.data import DataSet
from torch.utils.data import DataLoader

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
timesteps = 1000
batch_size = 64
lr = 5e-4
epochs = 10


In [ ]:
dataset = DataSet()
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    pin_memory=True,
    drop_last=True,
)

In [ ]:
model = DDPM().to(device)

gaussian_diffusion = GaussianDiffusion(timesteps)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

loss_func = Loss()

In [ ]:
# res应当用sample
for epoch in range(epochs):
    for step, (ori, tar) in enumerate(dataloader):
        optimizer.zero_grad()

        batch_size = ori.shape[0]

        ori = ori.to(device)
        tar = tar.to(device)

        noise = torch.randn_like(ori)

        t = torch.randint(0, timesteps, (batch_size,), device=device).long()

        ori_noise = gaussian_diffusion.q_sample(ori, t, noise=noise)

        res = model(noise, t)

        loss = loss_func.forward_loss(res, tar)

        loss.backward()
        optimizer.step()

        if step % 200 == 0:
            print("Loss:", loss.item())
